# <div align="center"> 图像分类 </div>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_post_cv_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_del_data
from k12libs.utils.nb_easy import k12ai_out_data
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_count_down

import os, time

In [3]:
wait_cnt = 30
wait_ckpt = True

## 1 CIFAR10

In [4]:
from k12libs.templates.cls.cifar10 import IC_CIFAR10_CONFIG

def cifar10_train_stop():
    cv_stop = k12ai_post_cv_request(
        uri='k12ai/framework/train',
        op='train.stop',
        user='test',
        uuid='cifar10',
    )
    k12ai_print(cv_stop)
    return cv_stop
    
def cifar10_train_start():
    cv_start = k12ai_post_cv_request(
        uri='k12ai/framework/train',
        op='train.start',
        user='test',
        uuid='cifar10',
        params=config
    )
    k12ai_print(cv_start)
    return cv_start

def cifar10_evaluate_stop():
    cv_stop = k12ai_post_cv_request(
        uri='k12ai/framework/evaluate',
        op='evaluate.stop',
        user='test',
        uuid='cifar10',
    )
    k12ai_print(cv_stop)
    return cv_stop

def cifar10_evaluate_start():
    cv_start = k12ai_post_cv_request(
        uri='k12ai/framework/evaluate',
        op='evaluate.start',
        user='test',
        uuid='cifar10',
        params=config
    )
    k12ai_print(cv_start)
    return cv_start

### 1.1 训练

In [5]:
config = IC_CIFAR10_CONFIG
config['solver']['max_epoch'] = 2

### train.stop

In [6]:
stop_context = cifar10_train_stop()

{
    "req": {
        "op": "train.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "cifar10",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/cifar10/train.stop"
}


In [7]:
k12ai_del_data(stop_context['key'])
k12ai_print(k12ai_get_data(stop_context['key'], 'error', waitcnt=3))

[
    {
        "key": "framework/test/cifar10/train.stop/error/202001032148",
        "value": {
            "version": "0.1.0",
            "type": "error",
            "tag": "framework",
            "op": "train.stop",
            "user": "test",
            "service_uuid": "cifar10",
            "timestamp": 1578059339714,
            "datetime": "20200103214859",
            "error": {
                "code": 100300,
                "descr": "success",
                "message": "container name:train-test-cifar10"
            }
        }
    }
]


### train.start

In [8]:
start_context = cifar10_train_start()

{
    "req": {
        "op": "train.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "cifar10",
        "service_params": {
            "dataset": "default",
            "task": "cls",
            "method": "image_classifier",
            "data": {
                "num_classes": 10,
                "data_dir": "/data/datasets/cv/cifar10",
                "image_tool": "pil",
                "input_mode": "RGB",
                "workers": 1,
                "mean_value": [
                    124,
                    116,
                    104
                ],
                "normalize": {
                    "div_value": 255,
                    "mean": [
                        0.485,
                        0.456,
                        0.406
                    ],
                    "std": [
                        0.229,
                        0.224,
                        0.225
                    ]
                }
            },


In [9]:
k12ai_del_data(start_context['key'])
if start_context['res']['code'] != 100000:
    k12ai_print(start_context['res'])
    raise 

In [10]:
k12ai_out_data(start_context['key'], ['status', 'metrics', 'error'], hook_func=k12ai_count_down(wait_cnt))

In [11]:
cifar10_train_stop()
k12ai_print(k12ai_get_data(start_context['key'], 'status', waitcnt=5))

{
    "req": {
        "op": "train.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "cifar10",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/cifar10/train.stop"
}
[
    {
        "key": "framework/test/cifar10/train.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "cifar10",
            "timestamp": 1578059344007,
            "datetime": "20200103214904",
            "status": {
                "value": "running"
            }
        }
    }
]


异步输出样例:

```json
{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "abcdef",
    "timestamp": 1574939332062,
    "datetime": "2019-11-28 19:08:52",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 2.3058,
        "training_speed": 0.105,
        "lr": [
            0.1,
            0.1
        ],
        "validation_loss": 2.3041,
        "validation_accuracy": 0.10519999710842967,
        "validation_accuracy3": 0.3018999940156937,
        "validation_accuracy5": 0.49709998607635497
    }
}
```

### 1.2 评估

### evaluate.stop

In [12]:
stop_context = cifar10_evaluate_stop()

{
    "req": {
        "op": "evaluate.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "cifar10",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/cifar10/evaluate.stop"
}


In [13]:
k12ai_del_data(stop_context['key'])
k12ai_print(k12ai_get_data(stop_context['key'], 'error', waitcnt=5))

### evaluate.start

In [14]:
start_context = cifar10_evaluate_start()

{
    "req": {
        "op": "evaluate.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "cifar10",
        "service_params": {
            "dataset": "default",
            "task": "cls",
            "method": "image_classifier",
            "data": {
                "num_classes": 10,
                "data_dir": "/data/datasets/cv/cifar10",
                "image_tool": "pil",
                "input_mode": "RGB",
                "workers": 1,
                "mean_value": [
                    124,
                    116,
                    104
                ],
                "normalize": {
                    "div_value": 255,
                    "mean": [
                        0.485,
                        0.456,
                        0.406
                    ],
                    "std": [
                        0.229,
                        0.224,
                        0.225
                    ]
                }
            

In [15]:
k12ai_del_data(start_context['key'])
if start_context['res']['code'] != 100000:
    k12ai_print(start_context['res'])
    raise 

In [16]:
k12ai_print(k12ai_get_data(start_context['key'], 'metrics', waitcnt=wait_cnt))
k12ai_print(k12ai_get_data(start_context['key'], 'error'))
k12ai_print(k12ai_get_data(start_context['key'], 'status'))

[
    {
        "key": "framework/test/cifar10/evaluate.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "evaluate.start",
            "user": "test",
            "service_uuid": "cifar10",
            "timestamp": 1578059421146,
            "datetime": "20200103215021",
            "status": {
                "value": "running"
            }
        }
    }
]


## 2 MNIST

### 2.1 训练

In [17]:
from k12libs.templates.cls.mnist import IC_MNIST_CONFIG

def mnist_train_stop():
    cv_stop = k12ai_post_cv_request(
        uri='k12ai/framework/train',
        op='train.stop',
        user='test',
        uuid='mnist',
    )
    k12ai_print(cv_stop)
    return cv_stop
    
def mnist_train_start():
    cv_start = k12ai_post_cv_request(
        uri='k12ai/framework/train',
        op='train.start',
        user='test',
        uuid='mnist',
        params=config
    )
    k12ai_print(cv_start)
    return cv_start

def mnist_evaluate_stop():
    cv_stop = k12ai_post_cv_request(
        uri='k12ai/framework/evaluate',
        op='evaluate.stop',
        user='test',
        uuid='mnist',
    )
    k12ai_print(cv_stop)
    return cv_stop

def mnist_evaluate_start():
    cv_start = k12ai_post_cv_request(
        uri='k12ai/framework/evaluate',
        op='evaluate.start',
        user='test',
        uuid='mnist',
        params=config
    )
    k12ai_print(cv_start)
    return cv_start

In [18]:
config = IC_MNIST_CONFIG
config['solver']['max_epoch'] = 2
config['network']['backbone'] = 'resnet18'
config['solver']['optim']['optim_method'] = 'adam'
config['train']['batch_size'] = 32

### train.stop

In [19]:
stop_context = mnist_train_stop()

{
    "req": {
        "op": "train.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/mnist/train.stop"
}


In [20]:
k12ai_del_data(stop_context['key'])
k12ai_print(k12ai_get_data(stop_context['key'], 'error', waitcnt=3))

[
    {
        "key": "framework/test/mnist/train.stop/error/202001032150",
        "value": {
            "version": "0.1.0",
            "type": "error",
            "tag": "framework",
            "op": "train.stop",
            "user": "test",
            "service_uuid": "mnist",
            "timestamp": 1578059449249,
            "datetime": "20200103215049",
            "error": {
                "code": 100300,
                "descr": "success",
                "message": "container name:train-test-mnist"
            }
        }
    }
]


### train.sart

In [21]:
start_context = mnist_train_start()

{
    "req": {
        "op": "train.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {
            "dataset": "default",
            "task": "cls",
            "method": "image_classifier",
            "data": {
                "num_classes": 10,
                "data_dir": "/data/datasets/cv/mnist",
                "image_tool": "cv2",
                "input_mode": "BGR",
                "workers": 1,
                "mean_value": [
                    124,
                    116,
                    104
                ],
                "normalize": {
                    "div_value": 1,
                    "mean": [
                        0.1307,
                        0.1307,
                        0.1307
                    ],
                    "std": [
                        0.3081,
                        0.3081,
                        0.3081
                    ]
                }
            },


In [22]:
k12ai_del_data(start_context['key'])
if start_context['res']['code'] != 100000:
    k12ai_print(start_context['res'])
    raise 

In [23]:
k12ai_print(k12ai_get_data(start_context['key'], 'metrics', waitcnt=wait_cnt))
k12ai_print(k12ai_get_data(start_context['key'], 'error'))
k12ai_print(k12ai_get_data(start_context['key'], 'status'))

[
    {
        "key": "framework/test/mnist/train.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "mnist",
            "timestamp": 1578059453594,
            "datetime": "20200103215053",
            "status": {
                "value": "running"
            }
        }
    }
]


In [24]:
mnist_train_stop()
k12ai_print(k12ai_get_data(start_context['key'], 'status', waitcnt=3))

{
    "req": {
        "op": "train.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/mnist/train.stop"
}
[
    {
        "key": "framework/test/mnist/train.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "mnist",
            "timestamp": 1578059453594,
            "datetime": "20200103215053",
            "status": {
                "value": "running"
            }
        }
    }
]


## 2.2 评估

### evaluate.stop

In [25]:
stop_context = mnist_evaluate_stop()

{
    "req": {
        "op": "evaluate.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/mnist/evaluate.stop"
}


In [26]:
k12ai_del_data(stop_context['key'])
k12ai_print(k12ai_get_data(stop_context['key'], 'error', waitcnt=3))

### evaluate.start

In [27]:
start_context = mnist_evaluate_start()

{
    "req": {
        "op": "evaluate.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {
            "dataset": "default",
            "task": "cls",
            "method": "image_classifier",
            "data": {
                "num_classes": 10,
                "data_dir": "/data/datasets/cv/mnist",
                "image_tool": "cv2",
                "input_mode": "BGR",
                "workers": 1,
                "mean_value": [
                    124,
                    116,
                    104
                ],
                "normalize": {
                    "div_value": 1,
                    "mean": [
                        0.1307,
                        0.1307,
                        0.1307
                    ],
                    "std": [
                        0.3081,
                        0.3081,
                        0.3081
                    ]
                }
            

In [28]:
k12ai_del_data(start_context['key'])
if start_context['res']['code'] != 100000:
    k12ai_print(start_context['res']['message'])
    raise 

In [29]:
k12ai_print(k12ai_get_data(start_context['key'], 'metrics', waitcnt=wait_cnt))
k12ai_print(k12ai_get_data(start_context['key'], 'error'))
k12ai_print(k12ai_get_data(start_context['key'], 'status'))

[
    {
        "key": "framework/test/mnist/evaluate.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "evaluate.start",
            "user": "test",
            "service_uuid": "mnist",
            "timestamp": 1578059488051,
            "datetime": "20200103215128",
            "status": {
                "value": "running"
            }
        }
    }
]


In [30]:
mnist_evaluate_stop()
k12ai_print(k12ai_get_data(start_context['key'], 'status', waitcnt=3))

{
    "req": {
        "op": "evaluate.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "mnist",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/mnist/evaluate.stop"
}
[
    {
        "key": "framework/test/mnist/evaluate.start/status",
        "value": {
            "version": "0.1.0",
            "type": "status",
            "tag": "framework",
            "op": "evaluate.start",
            "user": "test",
            "service_uuid": "mnist",
            "timestamp": 1578059488051,
            "datetime": "20200103215128",
            "status": {
                "value": "running"
            }
        }
    }
]
